In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

from sklearn.impute import KNNImputer

In [57]:
orders = pd.read_csv('orders_with_sequencesv2.csv', index_col=0)
nodes = pd.read_csv('nodes.csv')
nodes_final = pd.read_csv('nodes_test.csv')
nodes_info = pd.read_csv('nodes_info_clustered.csv')
df_train = pd.read_csv('final_df_start_center_finish.csv', index_col=0)

order_test = pd.read_csv('final_test_with_sequencesv2.csv', index_col=0)
df_test = pd.read_csv('Really_final_test_with_clustering_start_center_finish.csv')

orders.shape, df_train.shape, order_test.shape, df_test.shape

((5000, 6), (4997, 118), (1000, 4), (1000, 120))

In [12]:
df_train.head()

,Id,running_time,completed_time,route_distance_km,delta_time,is_afternoon,is_evening,is_morning,is_night,morning_peak_time,...,center_cluster_20,center_cluster_21,center_cluster_22,center_cluster_23,center_cluster_24,center_cluster_25,center_cluster_26,center_cluster_27,center_cluster_28,center_cluster_29
0,-4773019581999572651,2022-01-24 18:30:21,2022-01-24 18:44:43,3.740,862.0,0,1,0,0,0,...,0.000187,0.000089,0.000352,0.000223,0.000361,0.003389,0.001009,0.965341,0.000146,0.000343
1,-7575630690398473489,2022-01-24 06:53:53,2022-01-24 07:06:26,3.526,753.0,0,0,1,0,0,...,0.000727,0.000474,0.001438,0.000926,0.001176,0.006062,0.002630,0.028300,0.000603,0.002082
2,-6264582368520213833,2022-01-24 10:00:59,2022-01-24 10:15:58,5.071,899.0,0,0,1,0,0,...,0.002052,0.001067,0.003604,0.002392,0.003860,0.025585,0.009700,0.232100,0.001619,0.003763
3,5964315354301636538,2022-01-24 14:28:05,2022-01-24 14:35:08,2.867,423.0,1,0,0,0,0,...,0.000781,0.000653,0.000958,0.000814,0.001092,0.001724,0.001445,0.002135,0.000686,0.001092
4,1372379574816145639,2022-01-24 11:57:29,2022-01-24 12:06:29,3.751,540.0,0,0,1,0,0,...,0.001406,0.002246,0.003671,0.002517,0.001294,0.002202,0.001769,0.002392,0.001492,0.896434


In [13]:
orders.head()

,Id,running_time,completed_time,route_distance_km,delta_time,sequence
0,-4773019581999572651,2022-01-24 18:30:21,2022-01-24 18:44:43,3.740,862.0,"[4441087529, 4441087526, 4325099311, 290891853..."
1,-7575630690398473489,2022-01-24 06:53:53,2022-01-24 07:06:26,3.526,753.0,"[4790757673, 4790757674, 4790757672, 290404245..."
2,-6264582368520213833,2022-01-24 10:00:59,2022-01-24 10:15:58,5.071,899.0,"[6806370965, 6806370964, 6806370963, 680637096..."
3,5964315354301636538,2022-01-24 14:28:05,2022-01-24 14:35:08,2.867,423.0,"[988709862, 2072353444, 988709861, 4382736188,..."
4,1372379574816145639,2022-01-24 11:57:29,2022-01-24 12:06:29,3.751,540.0,"[5216935102, 5216935103, 1973798474, 197379847..."


In [14]:
df_test.head()

,Id,running_time,route_distance_km,sequence,is_afternoon,is_evening,is_morning,is_night,morning_peak_time,evening_peak_time,...,center_cluster_20,center_cluster_21,center_cluster_22,center_cluster_23,center_cluster_24,center_cluster_25,center_cluster_26,center_cluster_27,center_cluster_28,center_cluster_29
0,6198,2022-01-24 03:38:30,4.744,"[4773237812, 4773237811, 2059504508, 205950375...",0,0,0,1,0,0,...,0.013717,0.003685,0.045085,0.019603,0.024870,0.101205,0.069110,0.046614,0.009976,0.018058
1,6417,2022-01-24 03:45:51,6.279,"[5020979245, 290008230, 703126946, 290008229, ...",0,0,0,1,0,0,...,0.004839,0.001510,0.010368,0.005796,0.011196,0.590644,0.052000,0.075470,0.003472,0.006112
2,7054,2022-01-24 03:52:14,3.934,"[4363705737, 4362696212, 317184766, 4768027647...",0,0,0,1,0,0,...,0.002377,0.001551,0.005112,0.003165,0.003691,0.018325,0.008171,0.056137,0.001991,0.008026
3,9628,2022-01-24 04:03:21,5.959,"[3292022393, 3292022394, 701988912, 27126496, ...",0,0,0,1,0,0,...,0.029077,0.001871,0.546919,0.073559,0.018087,0.009697,0.014032,0.005721,0.017462,0.006324
4,10283,2022-01-24 04:01:12,7.028,"[1920586755, 1920586759, 2350039045, 392526370...",0,0,0,1,0,0,...,0.034722,0.002592,0.326604,0.063083,0.032142,0.017629,0.026947,0.009693,0.019732,0.009098


In [15]:
order_test.head()

,Id,running_time,route_distance_km,sequence
0,6198,2022-01-24 03:38:30,4.744,"[4773237812, 4773237811, 2059504508, 205950375..."
1,6417,2022-01-24 03:45:51,6.279,"[5020979245, 290008230, 703126946, 290008229, ..."
2,7054,2022-01-24 03:52:14,3.934,"[4363705737, 4362696212, 317184766, 4768027647..."
3,9628,2022-01-24 04:03:21,5.959,"[3292022393, 3292022394, 701988912, 27126496, ..."
4,10283,2022-01-24 04:01:12,7.028,"[1920586755, 1920586759, 2350039045, 392526370..."


In [24]:
order_test[order_test['Id'].isin(df_test['Id'])].count()

Id                   1000
running_time         1000
route_distance_km    1000
sequence             1000
dtype: int64

# Train

In [58]:
df_train['running_time'] = pd.to_datetime(df_train['running_time'])
df_train['hour'] = df_train['running_time'].dt.hour
df_train = df_train.drop(['running_time', 'completed_time'], axis=1)

In [59]:
# norm_columns = ['route_distance_km', 'temperature_2m (°C)', 'relativehumidity_2m (%)', 'apparent_temperature (°C)', 'snowfall (cm)', 
#                 'dewpoint_2m (°C)', 'cloudcover (%)', 'windspeed_10m (km/h)', 'windgusts_10m (km/h)', 'start_node_lat', 'start_node_lon', 
#                 'end_node_lat', 'end_node_lon', 'distance_haversine', 'distance_dummy_manhattan', 'direction', 'central_lon', 'central_lat']

# for col in norm_columns:
#     df_train[col] = (df_train[col] - df_train[col].mean()) / df_train[col].std()

In [60]:
nodes['pred_time'] = nodes['distance']/(nodes['speed']/3.6)
df_train['pred_time'] = df_train['Id'].map(nodes.groupby('Id')['pred_time'].sum())

In [61]:
df_train.to_csv('df_train.csv', index=False)

# Test

In [62]:
set(df_test.columns.values) - set(df_train.columns.values)

{'node_center', 'node_finish', 'node_start', 'running_time', 'sequence'}

In [63]:
df_test['running_time'] = pd.to_datetime(df_test['running_time'])
df_test['hour'] = df_test['running_time'].dt.hour
df_test = df_test.drop(['running_time', 'sequence', 'node_center', 'node_finish', 'node_start'], axis=1)

In [64]:
nodes_final

,Id,node_start,node_finish,distance,speed
0,6198,8952394129,8952394128,138.795710,37.0
1,6198,2059503754,4548172320,95.273001,33.0
2,6198,2059504508,2059503754,137.647881,35.0
3,6198,1570776534,1977018578,4.383708,30.0
4,6198,1977018580,1977018576,24.195593,31.0
...,...,...,...,...,...
102491,527850,290891780,5957304897,40.933260,45.0
102492,527850,5957304897,3902949792,63.421598,34.0
102493,527850,3902949792,290404192,4.303810,16.0
102494,527850,290404192,5957304888,5.250640,37.0


In [65]:
nodes_final['pred_time'] = nodes_final['distance']/(nodes_final['speed']/3.6)
df_test['pred_time'] = df_test['Id'].map(nodes_final.groupby('Id')['pred_time'].sum())

In [66]:
df_test

,Id,route_distance_km,is_afternoon,is_evening,is_morning,is_night,morning_peak_time,evening_peak_time,temperature_2m (°C),relativehumidity_2m (%),...,center_cluster_22,center_cluster_23,center_cluster_24,center_cluster_25,center_cluster_26,center_cluster_27,center_cluster_28,center_cluster_29,hour,pred_time
0,6198,4.744,0,0,0,1,0,0,-3.3,85,...,0.045085,0.019603,0.024870,0.101205,0.069110,0.046614,0.009976,0.018058,3,391.092968
1,6417,6.279,0,0,0,1,0,0,-3.3,85,...,0.010368,0.005796,0.011196,0.590644,0.052000,0.075470,0.003472,0.006112,3,524.819710
2,7054,3.934,0,0,0,1,0,0,-3.3,85,...,0.005112,0.003165,0.003691,0.018325,0.008171,0.056137,0.001991,0.008026,3,387.661475
3,9628,5.959,0,0,0,1,0,0,-3.6,84,...,0.546919,0.073559,0.018087,0.009697,0.014032,0.005721,0.017462,0.006324,4,561.546514
4,10283,7.028,0,0,0,1,0,0,-3.6,84,...,0.326604,0.063083,0.032142,0.017629,0.026947,0.009693,0.019732,0.009098,4,654.753300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,525706,2.897,0,1,0,0,0,1,-3.1,83,...,0.256855,0.080690,0.018308,0.019381,0.020553,0.014207,0.025768,0.033631,18,286.361660
996,526604,3.482,0,1,0,0,0,1,-3.1,83,...,0.041832,0.016570,0.009588,0.018551,0.015418,0.014385,0.007489,0.024628,18,356.409807
997,527213,3.486,0,1,0,0,0,1,-3.1,83,...,0.002636,0.001745,0.002349,0.011846,0.005221,0.062854,0.001166,0.003596,18,360.807126
998,527520,0.703,0,1,0,0,0,1,-3.1,83,...,0.004433,0.002893,0.005089,0.045375,0.014340,0.450485,0.001938,0.004059,18,76.890623


In [67]:
col_list = list(df_train.columns.values)
col_list.remove('delta_time')
df_test = df_test[col_list]

In [68]:
df_test.to_csv('df_test.csv', index=False)

# RNN Train

In [27]:
orders = pd.read_csv('orders.csv')
nodes = pd.read_csv('nodes.csv')

In [28]:
orders[orders['Id'].isin(nodes['Id'].unique())].shape

(5000, 5)

In [29]:
orders['running_time'] = pd.to_datetime(orders['running_time'])
orders['hour'] = orders['running_time'].dt.hour

In [30]:
df_nodes = pd.read_csv('nodes_info.csv')

# верхня права - 46.618659, 30.862736
# нижня ліва - 46.296001, 30.511882
max_lon = 30.862736
min_lon = 30.511882
max_lat = 46.618659
min_lat = 46.296001

df_nodes['check'] = ((df_nodes['lon'] > min_lon) & (df_nodes['lon'] < max_lon) & (df_nodes['lat'] > min_lat) & (df_nodes['lat'] < max_lat))
df_nodes['check'].value_counts()

True     19371
False      997
Name: check, dtype: int64

In [31]:
nodes = nodes.merge(df_nodes.loc[df_nodes['check'] == True,
                                ['node_id', 'lat', 'lon']], left_on='node_start', right_on='node_id')
nodes = nodes.merge(df_nodes.loc[df_nodes['check'] == True, 
                                 ['node_id', 'lat', 'lon']], left_on='node_finish', right_on='node_id', suffixes=['_s', '_f'])

nodes = nodes.drop(['node_start', 'node_finish', 'node_id_s', 'node_id_f'], axis=1)

nodes = nodes.merge(orders[['Id', 'hour', 'route_distance_km', 'delta_time']], on='Id')

nodes.head()

,Id,distance,speed,lat_s,lon_s,lat_f,lon_f,hour,route_distance_km,delta_time
0,-6374252502568484586,17.414917,25.0,46.472665,30.739248,46.472682,30.739022,10,2.156,436.0
1,-6374252502568484586,17.186843,27.0,46.472507,30.741320,46.472524,30.741097,10,2.156,436.0
2,-6374252502568484586,25.777868,32.0,46.472813,30.737195,46.472835,30.736860,10,2.156,436.0
3,-6374252502568484586,2.690220,33.0,46.467890,30.736439,46.488425,30.711088,10,2.156,436.0
4,-6374252502568484586,83.792561,35.0,46.473025,30.734244,46.473103,30.733156,10,2.156,436.0


In [32]:
knn_imputer = KNNImputer()
knn = knn_imputer.fit_transform(nodes)
knn

array([[-6.37425250e+18,  1.74149170e+01,  2.50000000e+01, ...,
         1.00000000e+01,  2.15600000e+00,  4.36000000e+02],
       [-6.37425250e+18,  1.71868430e+01,  2.70000000e+01, ...,
         1.00000000e+01,  2.15600000e+00,  4.36000000e+02],
       [-6.37425250e+18,  2.57778680e+01,  3.20000000e+01, ...,
         1.00000000e+01,  2.15600000e+00,  4.36000000e+02],
       ...,
       [ 6.45499002e+18,  8.04210740e+01,  2.98000000e+01, ...,
         1.80000000e+01,  2.19300000e+00,  5.48000000e+02],
       [ 6.45499002e+18,  1.33735509e+02,  2.98000000e+01, ...,
         1.80000000e+01,  2.19300000e+00,  5.48000000e+02],
       [ 6.45499002e+18,  6.11308480e+01,  1.80000000e+01, ...,
         1.80000000e+01,  2.19300000e+00,  5.48000000e+02]])

In [33]:
merged_data = pd.read_csv('merged_data.csv')

nodes = nodes.merge(merged_data[['Id', 'is_afternoon', 'is_evening', 'is_morning', 'is_night', 'morning_peak_time', 'evening_peak_time',
                                 'temperature_2m (°C)', 'relativehumidity_2m (%)', 'apparent_temperature (°C)', 'snowfall (cm)', 
                                  'dewpoint_2m (°C)', 'cloudcover (%)', 'windspeed_10m (km/h)', 'windgusts_10m (km/h)']], on='Id')

In [34]:
nodes['speed'] = knn[:, 2].astype(int)

norm_columns = ['distance', 'speed', 'lat_s', 'lon_s', 'lat_f', 'lon_f', 'route_distance_km',
                'temperature_2m (°C)', 'relativehumidity_2m (%)', 'apparent_temperature (°C)', 'snowfall (cm)', 
                'dewpoint_2m (°C)', 'cloudcover (%)', 'windspeed_10m (km/h)', 'windgusts_10m (km/h)']

# for col in norm_columns:
#     nodes[col] = (nodes[col] - nodes[col].mean()) / nodes[col].std()
    
nodes['delta_time'] = nodes['delta_time'].astype(int)

In [35]:
nodes.head()

,Id,distance,speed,lat_s,lon_s,lat_f,lon_f,hour,route_distance_km,delta_time,...,morning_peak_time,evening_peak_time,temperature_2m (°C),relativehumidity_2m (%),apparent_temperature (°C),snowfall (cm),dewpoint_2m (°C),cloudcover (%),windspeed_10m (km/h),windgusts_10m (km/h)
0,-6374252502568484586,17.414917,25,46.472665,30.739248,46.472682,30.739022,10,2.156,436,...,0,0,-2.8,73,-10.4,0.07,-7.0,100,32.1,52.6
1,-6374252502568484586,17.186843,27,46.472507,30.741320,46.472524,30.741097,10,2.156,436,...,0,0,-2.8,73,-10.4,0.07,-7.0,100,32.1,52.6
2,-6374252502568484586,25.777868,32,46.472813,30.737195,46.472835,30.736860,10,2.156,436,...,0,0,-2.8,73,-10.4,0.07,-7.0,100,32.1,52.6
3,-6374252502568484586,2.690220,33,46.467890,30.736439,46.488425,30.711088,10,2.156,436,...,0,0,-2.8,73,-10.4,0.07,-7.0,100,32.1,52.6
4,-6374252502568484586,83.792561,35,46.473025,30.734244,46.473103,30.733156,10,2.156,436,...,0,0,-2.8,73,-10.4,0.07,-7.0,100,32.1,52.6


In [37]:
nodes.to_csv('df_prepared_rnn_v3.csv')